In [1]:
from qutip import *
from tqdm import tqdm
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import math

In [2]:
from eigenfns import *
from arch import *

Code for OTOC

In [3]:
#for each T value
def cal_Aq(states_list,a,evol_op):
    #this is log of Ct thing
    A = []
    for i in range(len(a)):
        A_temp = 0
        x_op = (a[i] + a[i].dag())/2
        y_op = (a[i] - a[i].dag())/(2*1j)
        x_op_t = evol_op * x_op * evol_op.dag()
        op_com = commutator(x_op_t, y_op, 'normal')
        op_com_sq = op_com * op_com.dag()
        for state in states_list:
            A_temp += math.log(expect(op_com_sq,state))
        A.append(A_temp)
    return A

def cal_Aq_alt(states_list,a,evol_op):
    #this is log of Ct thing
    #interchange q->p
    A = []
    for i in range(len(a)):
        A_temp = 0
        x_op = (a[i] + a[i].dag())/2
        y_op = (a[i] - a[i].dag())/(2*1j)
        y_op_t = evol_op * y_op * evol_op.dag()
        op_com = commutator(y_op_t, x_op, 'normal')
        op_com_sq = op_com * op_com.dag()
        for state in states_list:
            A_temp += math.log(expect(op_com_sq,state))
        A.append(A_temp)
    return A


def cal_Aq_ind(states_list,a,evol_op,i = 1):
    #this is log of Ct thing
    #calculate for just 1 index
    A = []
    A_temp = 0
    x_op = (a[i] + a[i].dag())/2
    y_op = (a[i] - a[i].dag())/(2*1j)
    x_op_t = evol_op * x_op * evol_op.dag()
    op_com = commutator(x_op_t, y_op, 'normal')
    op_com_sq = op_com * op_com.dag()
    for state in states_list:
        A_temp += math.log(expect(op_com_sq,state))
    A.append(A_temp)
    return A


def cal_Aq_total(states_list,a,evol_op):
    #this is log of Ct thing
    #calculate for entire thing by taking tensor product
    A = []
    A_temp = 0
    x_op = (tensor(a) + tensor(a).dag())/2
    y_op = (a - a.dag())/(2*1j)
    x_op_t = evol_op * x_op * evol_op.dag()
    op_com = commutator(x_op_t, y_op, 'normal')
    op_com_sq = op_com * op_com.dag()
    for state in states_list:
        A_temp += math.log(expect(op_com_sq,state))
    A.append(A_temp)
    return A


#different calculation of OTOC
def cal_Aq2(states_list,a,evol_op):
    #this is log of Ct thing
    #calculate using a(t)'b'a(t)b thing
    A = []
    for i in range(len(a)):
        A_temp = 0
        x_op = (a[i] + a[i].dag())/2
        y_op = (a[i] - a[i].dag())/(2*1j)
        x_op_t = evol_op * x_op * evol_op.dag()
        op_otoc = x_op_t.dag()*y_op.dag()*x_op_t*y_op
        for state in states_list:
            A_temp += math.log(abs(expect(op_otoc,state)))
        A.append(A_temp)
    return A


In [4]:
def cal_otoc(run,timelist):
    tlist = run.tlist
    A_avg_arr = []
    A_max_arr = []
    
    for t in tqdm(range(len(tlist)),position=0):
        A_avg = []
        A_max = []
        H = run.t_arch.H_const + run.t_arch.H_int(tlist[t])
        states_list = run.c_evec[t]
        for time_step in tqdm(timelist,position=1):
            Q = (0+1j)*time_step*H
            evol_op = Q.expm()
            A = cal_Aq(states_list,run.t_arch.a,evol_op)
            A_avg.append(np.mean(A))
            A_max.append(max(A))
        A_avg_arr.append(A_avg)
        A_max_arr.append(A_max)
    
    return A_avg_arr, A_max_arr

In [5]:
tarch = transmon_arch(8,3)
tlist =np.linspace(0.1,2,3)
run = run_data(tarch,tlist)
run.cal_all()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:51<00:00, 77.11s/it]


In [6]:
timelist = np.linspace(0.1,1,10)
arr_avg, arr_max = cal_otoc(run,timelist)

100%|██████████████████████████████████████████████████████████████████████████████| 3/3 [18:02:56<00:00, 21658.94s/it]


In [ ]:
#setting up colors
x = np.arange(len(arr))
ys = [i+x+(i*x)**2 for i in range(len(x))]

colors = cm.rainbow(np.linspace(0, 1, len(ys)))

In [ ]:
#lyp exp calculation
lyp_arr_avg = []
for a in arr_avg:
    lyp_arr_avg.append(np.polyfit(timelist,a,1))
#lyp exp calculation
lyp_arr_max = []
for a in arr_max:
    lyp_arr_max.append(np.polyfit(timelist,a,1))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(0,0,label = str("coupling strength  ,  slope"))
for a,t,c,slope_arr in zip(arr_avg,tlist,colors,lyp_arr_avg):
    plt.plot(timelist,a, '-o', label = str(str(round(t,2)) +"   ,   "+str(round(slope_arr[0],2))),color = c)

title_temp = str(str(tarch.N) + " tranmons " + str(tarch.Nlevel)+ " level systems")
plt.title(str(title_temp)+"\n OTOC_avg")
plt.xlabel("time")
plt.ylabel("log(C(T))")
plt.legend(loc='best')


In [ ]:
plt.figure(figsize=(8,8))
plt.plot(0,0,label = str("coupling strength  ,  slope"))
for a,t,c,slope_arr in zip(arr_max,tlist,colors,lyp_arr_max):
    plt.plot(timelist,a, '-o', label = str(str(round(t,2)) +"   ,   "+str(round(slope_arr[0],2))),color = c)

title_temp = str(str(tarch.N) + " tranmons " + str(tarch.Nlevel)+ " level systems")
plt.title(str(title_temp)+"\n OTOC_max")
plt.xlabel("time")
plt.ylabel("log(C(T))")
plt.legend(loc='best')
